# Sending Insights to Remote Server

This example walks you through the API for sending insights to remote metrics servers.

## Prerequisites

This notebooks needs to be run in the `tempo-examples` conda environment defined below. Create from project root folder:

```bash
conda env create --name tempo-examples --file conda/tempo-examples.yaml
```

In [1]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [2]:
import os

ARTIFACTS_FOLDER = os.getcwd()+"/artifacts"
TEMPO_DIR = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))

### Custom inference logic 

Our custom model will be very simple to focus the logic on the insights functionality.


In [3]:
from tempo.serve.metadata import SeldonCoreOptions
from tempo.serve.constants import DefaultInsightsLocalEndpoint, DefaultInsightsK8sEndpoint

seldon_options = SeldonCoreOptions(**{
    "local_options": { "insights_options": { "worker_endpoint": DefaultInsightsLocalEndpoint } },
    "remote_options": {
        "insights_options": { "worker_endpoint": DefaultInsightsK8sEndpoint },
        "namespace": "seldon",
        "authSecretName": "minio-secret",
    },
})

In [4]:
import numpy as np
from tempo.serve.utils import pipeline, predictmethod
from tempo.magic import t

@pipeline(
    name='insights-pipeline',
    uri="s3://tempo/insights-pipeline/resources",
    local_folder=ARTIFACTS_FOLDER,
    runtime_options=seldon_options.local_options,
)
class Pipeline:
    
    @predictmethod
    def predict(self, data: np.ndarray, parameters: dict) -> np.ndarray:
        if parameters.get("log"):
            t.insights.log_request()
            t.insights.log_response()
            t.insights.log(parameters)
        return data


## Create pipeline

In [5]:
pipeline = Pipeline()

## Deploy Docker Insights Dumper

In [9]:
from tempo.docker.utils import deploy_insights_message_dumper

deploy_insights_message_dumper()

## Print logs to make sure there's none

In [10]:
from tempo.docker.utils import get_logs_insights_message_dumper

print(get_logs_insights_message_dumper())

### Explicitly Log Insights 
We explicitly request to log by passing the parameters

In [11]:
params = { "log": "value" }
data = np.array([63])
pred = pipeline(data, params)
print(pred)

Attempted to log request but called manager directly, see documentation [TODO]
Attempted to log response but called manager directly, see documentation [TODO]


[63]


## Check the logs

We can see that only the params has been logged. 

This is because even we are passing an input to our model, there is still no request or response. 

To log HTTP request/response, this is relevant when the model is deployed.

In [12]:
print(get_logs_insights_message_dumper())

-----------------
{
    "path": "/",
    "headers": {
        "host": "0.0.0.0:8080",
        "ce-id": "3254a0e9-d436-4355-b279-dbf44eb1f7de",
        "ce-specversion": "0.3",
        "ce-source": "io.seldon.serving.deployment.NOTIMPLEMENTED.NOTIMPLEMENTED",
        "ce-type": "io.seldon.serving.inference.custominsight",
        "ce-requestid": "3254a0e9-d436-4355-b279-dbf44eb1f7de",
        "ce-modelid": "NOTIMPLEMENTED",
        "ce-inferenceservicename": "NOTIMPLEMENTED",
        "ce-namespace": "NOTIMPLEMENTED",
        "ce-endpoint": "NOTIMPLEMENTED",
        "accept": "*/*",
        "accept-encoding": "gzip, deflate",
        "user-agent": "Python/3.7 aiohttp/3.6.2",
        "content-length": "16",
        "content-type": "application/json"
    },
    "method": "POST",
    "body": "{\"log\": \"value\"}",
    "fresh": false,
    "hostname": "0.0.0.0",
    "ip": "::ffff:172.18.0.1",
    "ips": [],
    "protocol": "http",
    "query": {},
    "subdomains": [],
    "xhr": false,
    

### Don't log insights

We are now going to send the values with the blank parameters to avoid passing through the branch that explicitly logs the insights.

In [13]:
params = { }
data = np.array([63])
pred = pipeline(data, params)
print(pred)

[63]


## Check logs again
We can see that now new logs have been added

In [14]:
print(get_logs_insights_message_dumper())

-----------------
{
    "path": "/",
    "headers": {
        "host": "0.0.0.0:8080",
        "ce-id": "3254a0e9-d436-4355-b279-dbf44eb1f7de",
        "ce-specversion": "0.3",
        "ce-source": "io.seldon.serving.deployment.NOTIMPLEMENTED.NOTIMPLEMENTED",
        "ce-type": "io.seldon.serving.inference.custominsight",
        "ce-requestid": "3254a0e9-d436-4355-b279-dbf44eb1f7de",
        "ce-modelid": "NOTIMPLEMENTED",
        "ce-inferenceservicename": "NOTIMPLEMENTED",
        "ce-namespace": "NOTIMPLEMENTED",
        "ce-endpoint": "NOTIMPLEMENTED",
        "accept": "*/*",
        "accept-encoding": "gzip, deflate",
        "user-agent": "Python/3.7 aiohttp/3.6.2",
        "content-length": "16",
        "content-type": "application/json"
    },
    "method": "POST",
    "body": "{\"log\": \"value\"}",
    "fresh": false,
    "hostname": "0.0.0.0",
    "ip": "::ffff:172.18.0.1",
    "ips": [],
    "protocol": "http",
    "query": {},
    "subdomains": [],
    "xhr": false,
    

### Deploy the  Model to Docker

Finally, we'll be able to deploy our model using Tempo against one of the available runtimes (i.e. Kubernetes, Docker or Seldon Deploy).

We'll deploy first to Docker to test.

In [6]:
%%writetemplate $ARTIFACTS_FOLDER/conda.yaml
name: tempo-insights
channels:
  - defaults
dependencies:
  - pip=21.0.1
  - python=3.7.9
  - pip:
    - mlops-tempo @ file://{TEMPO_DIR}
    - mlserver==0.3.1.dev7

In [7]:
from tempo.serve.loader import save
save(pipeline, save_env=True)

Packing environment at '/home/alejandro/miniconda3/envs/tempo-36a1d148-c285-415e-849c-e56e8e89a9cb' to '/home/alejandro/Programming/kubernetes/seldon/tempo/docs/examples/logging-insights/artifacts/environment.tar.gz'
[########################################] | 100% Completed | 13.8s


In [19]:
from tempo.serve.constants import DefaultInsightsDockerEndpoint
from tempo import deploy_local

seldon_options.local_options.insights_options.worker_endpoint = DefaultInsightsDockerEndpoint

remote_model = deploy_local(pipeline, seldon_options)

We can now test our model deployed in Docker as:

## Log insights

In [20]:
params = { "log": "value" }
data = np.array([63])
remote_model.predict(data=data, parameters=params)

array([63])

## Check that all logs are now present
Now we can see that in our model running in docker, the request and response were also logged as per our code logic

In [21]:
print(get_logs_insights_message_dumper())

-----------------
{
    "path": "/",
    "headers": {
        "host": "0.0.0.0:8080",
        "ce-id": "3254a0e9-d436-4355-b279-dbf44eb1f7de",
        "ce-specversion": "0.3",
        "ce-source": "io.seldon.serving.deployment.NOTIMPLEMENTED.NOTIMPLEMENTED",
        "ce-type": "io.seldon.serving.inference.custominsight",
        "ce-requestid": "3254a0e9-d436-4355-b279-dbf44eb1f7de",
        "ce-modelid": "NOTIMPLEMENTED",
        "ce-inferenceservicename": "NOTIMPLEMENTED",
        "ce-namespace": "NOTIMPLEMENTED",
        "ce-endpoint": "NOTIMPLEMENTED",
        "accept": "*/*",
        "accept-encoding": "gzip, deflate",
        "user-agent": "Python/3.7 aiohttp/3.6.2",
        "content-length": "16",
        "content-type": "application/json"
    },
    "method": "POST",
    "body": "{\"log\": \"value\"}",
    "fresh": false,
    "hostname": "0.0.0.0",
    "ip": "::ffff:172.18.0.1",
    "ips": [],
    "protocol": "http",
    "query": {},
    "subdomains": [],
    "xhr": false,
    

## Don't log

In [22]:
params = { }
data = np.array([63])
remote_model.predict(data=data, parameters=params)

array([63])

## Also we can see logs are not present when requested

When providing the explicit configuration the logs are not sent, and this can be seen in the insights logger container logs

In [23]:
print(get_logs_insights_message_dumper())

-----------------
{
    "path": "/",
    "headers": {
        "host": "0.0.0.0:8080",
        "ce-id": "3254a0e9-d436-4355-b279-dbf44eb1f7de",
        "ce-specversion": "0.3",
        "ce-source": "io.seldon.serving.deployment.NOTIMPLEMENTED.NOTIMPLEMENTED",
        "ce-type": "io.seldon.serving.inference.custominsight",
        "ce-requestid": "3254a0e9-d436-4355-b279-dbf44eb1f7de",
        "ce-modelid": "NOTIMPLEMENTED",
        "ce-inferenceservicename": "NOTIMPLEMENTED",
        "ce-namespace": "NOTIMPLEMENTED",
        "ce-endpoint": "NOTIMPLEMENTED",
        "accept": "*/*",
        "accept-encoding": "gzip, deflate",
        "user-agent": "Python/3.7 aiohttp/3.6.2",
        "content-length": "16",
        "content-type": "application/json"
    },
    "method": "POST",
    "body": "{\"log\": \"value\"}",
    "fresh": false,
    "hostname": "0.0.0.0",
    "ip": "::ffff:172.18.0.1",
    "ips": [],
    "protocol": "http",
    "query": {},
    "subdomains": [],
    "xhr": false,
    

## Now undeploy to move to Kubernetes

In [24]:
remote_model.undeploy()

In [25]:
from tempo.docker.utils import undeploy_insights_message_dumper

undeploy_insights_message_dumper()

## Deploy to Kubernetes with Tempo

 * Here we illustrate how the same workflow applies in kubernetes
 
### Prerequisites
 
 Create a Kind Kubernetes cluster with Minio and Seldon Core installed using Ansible from the Tempo project Ansible playbook.
 
 ```
 ansible-playbook ansible/playbooks/default.yaml
 ```

In [27]:
!kubectl create ns seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [28]:
!kubectl apply -f k8s/rbac -n seldon

secret/minio-secret created
serviceaccount/tempo-pipeline created
role.rbac.authorization.k8s.io/tempo-pipeline created
rolebinding.rbac.authorization.k8s.io/tempo-pipeline-rolebinding created


In [9]:
from tempo.examples.minio import create_minio_rclone
import os
create_minio_rclone(os.getcwd()+"/rclone.conf")

In [10]:
from tempo.serve.loader import upload
upload(pipeline)

In [11]:
from tempo.k8s.utils import deploy_insights_message_dumper

deploy_insights_message_dumper()

In [12]:
from tempo.k8s.utils import get_logs_insights_message_dumper

print(get_logs_insights_message_dumper())

In [18]:
from tempo import deploy_remote

remote_model = deploy_remote(pipeline, seldon_options)

## Log insights

In [19]:
params = { "log": "value" }
data = np.array([63])
remote_model.predict(data=data, parameters=params)

array([63])

In [15]:
from tempo.k8s.utils import get_logs_insights_message_dumper

print(get_logs_insights_message_dumper())

-----------------
{
    "path": "/",
    "headers": {
        "host": "insights-dumper.seldon-system:8080",
        "ce-id": "2180add5-ccdd-48a8-84fc-ce90d689e00e",
        "ce-specversion": "0.3",
        "ce-source": "io.seldon.serving.deployment.insights-pipeline.seldon",
        "ce-type": "io.seldon.serving.inference.custominsight",
        "ce-requestid": "2180add5-ccdd-48a8-84fc-ce90d689e00e",
        "ce-modelid": "insights-pipeline",
        "ce-inferenceservicename": "insights-pipeline",
        "ce-namespace": "seldon",
        "ce-endpoint": "default",
        "accept": "*/*",
        "accept-encoding": "gzip, deflate",
        "user-agent": "Python/3.7 aiohttp/3.7.4.post0",
        "content-length": "16",
        "content-type": "application/json"
    },
    "method": "POST",
    "body": "{\"log\": \"value\"}",
    "fresh": false,
    "hostname": "insights-dumper.seldon-system",
    "ip": "::ffff:10.1.15.244",
    "ips": [],
    "protocol": "http",
    "query": {},
    "su

## Don't log

In [21]:
params = {  }
data = np.array([63])
remote_model.predict(data=data, parameters=params)

array([63])

In [22]:
print(get_logs_insights_message_dumper())

-----------------
{
    "path": "/",
    "headers": {
        "host": "insights-dumper.seldon-system:8080",
        "ce-id": "2180add5-ccdd-48a8-84fc-ce90d689e00e",
        "ce-specversion": "0.3",
        "ce-source": "io.seldon.serving.deployment.insights-pipeline.seldon",
        "ce-type": "io.seldon.serving.inference.custominsight",
        "ce-requestid": "2180add5-ccdd-48a8-84fc-ce90d689e00e",
        "ce-modelid": "insights-pipeline",
        "ce-inferenceservicename": "insights-pipeline",
        "ce-namespace": "seldon",
        "ce-endpoint": "default",
        "accept": "*/*",
        "accept-encoding": "gzip, deflate",
        "user-agent": "Python/3.7 aiohttp/3.7.4.post0",
        "content-length": "16",
        "content-type": "application/json"
    },
    "method": "POST",
    "body": "{\"log\": \"value\"}",
    "fresh": false,
    "hostname": "insights-dumper.seldon-system",
    "ip": "::ffff:10.1.15.244",
    "ips": [],
    "protocol": "http",
    "query": {},
    "su

In [23]:
remote_model.undeploy()

In [24]:
from tempo.k8s.utils import undeploy_insights_message_dumper

undeploy_insights_message_dumper()